In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

# 네이버 영화 클레멘타인 평점크롤링

네이버 영화 클레멘타인에 유저들이 작성한 한줄평을 크롤링 해보겠습니다.

아래 굵은 글씨로 기술된 테스크들을 완수하세요.

**네이버 영화 클레멘타인 URL**

https://movie.naver.com/movie/bi/mi/point.nhn?code=37886

**1. 요청할 url 정의하기**

위 URL 페이지를 확인해보면 댓글평점 페이지가 넘어가도 URL이 변하지 않는 것을 확인 할 수 있습니다.

동적페이지 크롤링을 위해 개발자 도구로 숨겨진 URL을 확인 해봅시다.

우선 1페이지 크롤링을 해보겠습니다.

In [9]:
page = 1

**1. 숨은 url 설정 및 헤더 정보 정의**

In [49]:
# CODE
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=37886&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore"
header = {
    "referer" : "https://movie.naver.com/movie/bi/mi/point.naver?code=37886",
    "user-agent" : "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
}


**2. resp 변수명으로 requests로 url 요청한 데이터의 text를 저장합니다.**

In [50]:
# CODE
resp = requests.get(url)

**3. resp 결과값 앞 1000글자 확인 및 어떤형태의 데이터인지 파악하기**

In [51]:
# CODE
resp.text[:1001] # str, html


'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t<meta charset="utf-8">\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\t<title>네이버 영화</title>\r\n\t\n\n\n\t\n\t\n\t\n\t\n\t\t<link rel="shortcut icon" href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" type="image/x-icon">\n\t\t\t\n\n\r\n\t\n<link rel="stylesheet" type="text/css" href="/css/common.css?20220823121742" />\r\n\t\n<link rel="stylesheet" type="text/css" href="/css/movie_tablet.css?20220823121742" />\n<link rel="stylesheet" type="text/css" href="/css/movie_end.css?20220823121742" />\r\n\t\r\n\t\n\n\n<script type="text/javascript" src="/js/deploy/movie.all.js?20220823121742"></script>\n\n\n\r\n</head>\r\n<body>\r\n\t<!-- content -->\r\n<input type="hidden" name="movieCode" id="movieCode" value="37886"/>\r\n<input type="hidden" name="onlyActualPointYn" id="onlyActualPointYn" value="N"/>\r\n<input type="hidden" name="includeSpoilerYn" id="includeSpoilerYn" value="N"/>\r\n

'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<title>네이버 영화</title>\r\n\n\n\n\t\n\t\n\t\n\t\n\t\t<link rel="shortcut icon" href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" type="image/x-icon">\n\t\t\t\n\n\r\n\n<link rel="stylesheet" type="text/css" href="/css/common.css?20200629181825" />\r\n\n<link rel="stylesheet" type="text/css" href="/css/movie_tablet.css?20200629181825" />\n<link rel="stylesheet" type="text/css" href="/css/movie_end.css?20200629181825" />\r\n\r\n\n\n\n<script type="text/javascript" src="/js/deploy/movie.all.js?20200629181825"></script>\n\n\n\r\n</head>\r\n<body>\r\n\t<!-- content -->\r\n<input type="hidden" name="movieCode" id="movieCode" value="37886"/>\r\n<input type="hidden" name="onlyActualPointYn" id="onlyActualPointYn" value="N"/>\r\n<input type="hidden" name="includeSpoilerYn" id="includeSpoilerYn" value="N"/>\r\n<input type="hid

**4. response 결과에 따라 적절한 변환 함수를 사용하여 data라는 변수명에 저장합니다.**

In [52]:
# CODE

resp = requests.get(url, headers=header)
data = BeautifulSoup(resp.text, 'html.parser')

**5. 전설적인 댓글평점 '이 영화를 보고 암이 나았습니다.' 에 selector를 사용하여 접근해봅시다.**

In [61]:
# CODE
# #_filtered_ment_0
# data
# data.select('p')[4]
data.select('p span')[0]

<span id="_filtered_ment_0">
														
															
															
																이 영화를 보고 암이 나았습니다. 
															
														
														
													</span>

[<span id="_filtered_ment_0">
 														
 															
 															
 																이 영화를 보고 암이 나았습니다. 
 															
 														
 														
 													</span>]

**6. selector로 선택한 항목에서 텍스트만을 추려봅시다.**

In [69]:
# CODE
cancer = data.select('p span')[0].text
cancer = cancer.replace('\t', '')
cancer = cancer.replace('\n', '')
print(cancer)

'이 영화를 보고 암이 나았습니다.'

**7. 1페이지의 모든 숫자평점과, 댓글평점을 프린트해 봅시다.**

In [141]:
# CODE
star_list = []
data_list = []
for count in range(10):
    star_list.append(data.select('div.star_score em')[count].text)
    for item in data.select(f'li span#_filtered_ment_{count}'):
        data_list.append(item.text.strip().replace(',', ''))
    result = list(zip(star_list, data_list))
    print(result[count])


('10', '이 영화를 보고 암이 나았습니다.')
('1', '이것은절대1점이아니다11점을주고싶은 내마음이다')
('10', '모니터도 울고 외장하드도 울고 숨어있던 바이러스도 울었다')
('10', '당신이 이 영화를 보지 않았다면  아직 살아있을 이유 하나를 간직하고 있는 것이다.')
('10', '영화계엔 BC와 AC가 있다. Before Clementain After Clementain...')
('10', '남친 집에서 클레멘타인 DVD를 발견했고 결혼을 결심했습니다.')
('10', '드디어 보았네요. 나이 40대 중반에 이런 감동을 느낄 수 있음에 스스로도 대견한 생각이 듭니다. 먼 훗날 제 아이들도 이 영화를 보고 저와 같은 감동을 느끼길 바라는 건 그저 바램이겠지요?')
('10', '평점깍아내리는 낚시글 몇개있는데 제발그만해라 이기적인새끼들아좋은건 나누는거다')
('10', '장난으로 쓰지마라... 본인은 2004년에 실제로 극장에서 이 영화를 봤다')
('10', '프로포즈 선물로 다이아반지 대신 클레멘타인 파일을 USB에담아 목걸이로 만들어 그녀 목에 걸어주었다. 눈물로 대신 대답한 그녀...그리고 2세이름은 그녀의 의견을 반영해 이동준과 스티븐시갈의 이름을 딴 이동갈로 지을...')


10 이 영화를 보고 암이 나았습니다.
1 이것은절대1점이아니다11점을주고싶은 내마음이다
10 모니터도 울고 외장하드도 울고 숨어있던 바이러스도 울었다
10 당신이 이 영화를 보지 않았다면  아직 살아있을 이유 하나를 간직하고 있는 것이다.
10 영화계엔 BC와 AC가 있다. Before Clementain, After Clementain...
10 남친 집에서 클레멘타인 DVD를 발견했고, 결혼을 결심했습니다.
10 드디어 보았네요. 나이 40대 중반에 이런 감동을 느낄 수 있음에 스스로도 대견한 생각이 듭니다. 먼 훗날 제 아이들도 이 영화를 보고 저와 같은 감동을 느끼길 바라는 건 그저 바램이겠지요?
10 평점깍아내리는 낚시글 몇개있는데 제발그만해라 이기적인새끼들아좋은건 나누는거다
10 장난으로 쓰지마라... 본인은 2004년에 실제로 극장에서 이 영화를 봤다
10 프로포즈 선물로 다이아반지 대신 클레멘타인 파일을 USB에담아 목걸이로 만들어 그녀 목에 걸어주었다. 눈물로 대신 대답한 그녀...그리고 2세이름은 그녀의 의견을 반영해 이동준과 스티븐시갈의 이름을 딴 이동갈로 지을...


**8. 네이버에서는 기계적으로 접근하는 요청을 잘 차단합니다. 지난시간 차단을 막는 코드를 포함하여 1~10페이지의 평점과 댓글평점을 크롤링하여 데이터프레임으로 저장하세요**

In [146]:
# CODE
# 저장할 데이터프레임과 리스트 생성

data_list = []
star_list = []


# 반복문을 사용하여 페이지 URL 생성

for page in range(1, 11):
    url = f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=37886&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}"
    print(f"{page} 페이지 정상처리")

# 난수를 생성하여 차단 피하기
# requests 요청 전에 실행되도록 위치합니다.
    randn = np.random.randint(100)
    np.random.seed(randn)
    time_sleep = np.random.randint(1, 5)
    time.sleep(time_sleep)

# url 요청

    resp = requests.get(url, headers=header)
    resp.text
# 데이터 파싱
    star_list.append(data.select('div.star_score em')[page - 1].text)
    for item in data.select(f'li span#_filtered_ment_{page}'):
        data_list.append(item.text.strip().replace(',', ''))
    result = list(zip(star_list, data_list))
    print(result[page - 1])
# 반복문을 사용하여 리스트에 추출정보를 저장

# 리스트정보를 데이터프레임에 저장

1 페이지 정상처리
('10', '이것은절대1점이아니다11점을주고싶은 내마음이다')
2 페이지 정상처리
('1', '모니터도 울고 외장하드도 울고 숨어있던 바이러스도 울었다')
3 페이지 정상처리
('10', '당신이 이 영화를 보지 않았다면  아직 살아있을 이유 하나를 간직하고 있는 것이다.')
4 페이지 정상처리
('10', '영화계엔 BC와 AC가 있다. Before Clementain After Clementain...')
5 페이지 정상처리
('10', '남친 집에서 클레멘타인 DVD를 발견했고 결혼을 결심했습니다.')
6 페이지 정상처리
('10', '드디어 보았네요. 나이 40대 중반에 이런 감동을 느낄 수 있음에 스스로도 대견한 생각이 듭니다. 먼 훗날 제 아이들도 이 영화를 보고 저와 같은 감동을 느끼길 바라는 건 그저 바램이겠지요?')
7 페이지 정상처리
('10', '평점깍아내리는 낚시글 몇개있는데 제발그만해라 이기적인새끼들아좋은건 나누는거다')
8 페이지 정상처리
('10', '장난으로 쓰지마라... 본인은 2004년에 실제로 극장에서 이 영화를 봤다')
9 페이지 정상처리
('10', '프로포즈 선물로 다이아반지 대신 클레멘타인 파일을 USB에담아 목걸이로 만들어 그녀 목에 걸어주었다. 눈물로 대신 대답한 그녀...그리고 2세이름은 그녀의 의견을 반영해 이동준과 스티븐시갈의 이름을 딴 이동갈로 지을...')
10 페이지 정상처리


IndexError: list index out of range

1 페이지 정상처리
2 페이지 정상처리
3 페이지 정상처리
4 페이지 정상처리
5 페이지 정상처리
6 페이지 정상처리
7 페이지 정상처리
8 페이지 정상처리
9 페이지 정상처리
10 페이지 정상처리


,star,reply
0,10,이 영화를 보고 암이 나았습니다.
1,1,이것은절대1점이아니다11점을주고싶은 내마음이다
2,10,모니터도 울고 외장하드도 울고 숨어있던 바이러스도 울었다
3,10,당신이 이 영화를 보지 않았다면 아직 살아있을 이유 하나를 간직하고 있는 것이다.
4,10,"영화계엔 BC와 AC가 있다. Before Clementain, After Clem..."
...,...,...
95,10,도대체 3D 블루레이 버전은 언제나오나요... 기금모금이라도 하고 싶습니다.
96,10,이영화를 보고나서 아버지 방으로가서 주무시고 계시는 아버지의 얼굴을 물끄러미 쳐다봤...
97,10,1분이 1초 같아요
98,10,"밑의 분 너무하시네요. 이런 명작을 폄하하시다니, 이 영화를 보신적이 있으시기나 한..."


**9. 우리가 수집한 데이터를 바탕으로 이 영화를 추천할 수 있을까요? 여러분의 생각은 어떠한가요?**

In [ ]:
# ANSWER
